In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import numpy as np

def rand_gauss():
    """Generate a random number from a Gaussian distribution with mean 0 and std 1."""
    return np.random.normal()

def rand_number():
    """Generate a random number uniformly distributed between 0 and 1."""
    return np.random.rand()

def min_value(x):
    min_x = np.zeros(len(x)-1)
    for ii, x_i in enumerate(x[:-1]):
        min_x[ii] = abs(x[ii+1] - x[ii])
    mx = np.min(min_x)
    return mx
# uniform dx selection

def interpolation(x,y):
    mx = min_value(x) 
    dx = mx/100
    x_final = np.array([])
    y_final = np.array([])
    for ii, x_i in enumerate(x[:-1]):
        x_new = np.arange(x[ii],x[ii+1],dx) # x에서 mx가 최소가 되는 값의 / 100 을해서  dx 가 일정하게 맞춤.
        if ii >= len(x) - 2 :
            y_new = np.linspace(y[ii],y[ii+1],len(x_new))
        else :
            y_new = np.linspace(y[ii],y[ii+1],len(x_new),endpoint=False)
        x_final = np.append(x_final,x_new)
        y_final = np.append(y_final,y_new)
    return x_final, y_final
# Interpolation

def find_y(x,r_new,em_new): 
    dx = 0.0005
    y = np.where((r_new>= x-dx) & (r_new<= x+dx))[0]
    y_new=em_new[y].mean()
    return y_new


In [4]:
def gen_photon_Gaussian(photon, v_emit, atom, grid):
    """
    Generates a photon with a Gaussian velocity distribution in an emission region.

    Parameters:
    - photon: dict representing the photon properties (should have keys x, y, z, etc.)
    - v_emit: float, the standard deviation of the Gaussian velocity distribution
    - atom: dict containing atomic properties like frequencies and probabilities
    - grid: dict containing grid dimensions

    Returns:
    - photon: Updated dictionary with generated photon properties.
    """
    c = 3.0e8  # Speed of light in m/s
    pi = np.pi

    # Initial position
    phi = 2.0 * pi * rand_number()
    cost = 2.0 * rand_number() - 1.0  # Cosine of theta, uniformly distributed
    sint = np.sqrt(1.0 - cost**2)
    cosp = np.cos(phi)
    sinp = np.sin(phi)

    file = pd.read_csv('/home/jin/CIV_Photoionization/CIV_emissivity.txt',sep='\t',header=0)
    file_den = pd.read_csv('/home/jin/CIV_Photoionization/CIV_number_density.txt',sep='\t',header=0)

    radius,emiss= interpolation(file['Radius_kpc'].to_numpy(), file['Emissivity'].to_numpy())
    radius_d,num_den= interpolation(file_den['Radius_kpc'].to_numpy(), file_den['number_density'].to_numpy())

    Ro = 25

    ii = np.where(radius<=Ro )[0]
    # plt.plot(radius[ii],emiss[ii])
    radius_civ, emiss_civ,num_den_civ = radius[ii],emiss[ii], num_den[ii]

    Total_A = np.sum(emiss_civ)
    density = emiss_civ/Total_A
    # plt.plot(radius_civ,density,'ro')
    name=['x','PDF']
    tt = pd.DataFrame(np.column_stack((radius_civ,density)),columns=name)
    tt.to_csv('/home/jin/CIV_Photoionization/PDF.txt', sep='\t', index=False)

    radius_rand = []
    prob= []
    find_number_den=[]
    x = []
    y = []
    z = []
    reject_radius= []
    reject_prob =[]
    p_max = density.max()
    n , loop = 0 , 0 
    photon_number=50
    while (n<=photon_number):
        loop +=1 
        r1,r2 = np.random.random(2)
        r0 = r1*Ro
        px = find_y(r0,radius_civ,density)
        if (r2 <= px/p_max):
            n+=1
            radius_rand.append(r0)
            # prob.append(r2)
            theta = np.random.uniform(0, 2 * np.pi)
            phi = np.arccos(np.random.uniform(-1, 1))

            photon['x'] = r0*sint * cosp
            photon['y'] = r0*sint * sinp
            photon['z'] = r0*cost

            photon['ix'] = grid['N_X'] // 2 + 1
            photon['iy'] = grid['N_Y'] // 2 + 1
            photon['iz'] = grid['N_Z'] // 2 + 1

            photon['x_s'] = photon['x']
            photon['y_s'] = photon['y']
            photon['z_s'] = photon['z']

    # Random velocity in emission region
    # v_emit is sigma of Gaussian distribution
    vel = rand_gauss() * v_emit

    # Initial wavelength (frequency)
    temp = rand_number()
    if temp <= atom['f12_K'] / atom['f12']:
        photon['nu'] = atom['nuK'] / (1.0 - vel / c)
        photon['line'] = 1
    else:
        photon['nu'] = atom['nuH'] / (1.0 - vel / c)
        photon['line'] = 2

    # Call to initialize the photon (assume it's another function)
    initial_photon(photon)

    return photon

def initial_photon(photon):
    """Initialize photon properties. Placeholder function for customization."""
    # Add any necessary initialization logic here
    pass

# Example usage
photon = {}
atom = {'f12_K': 0.8, 'f12': 1.0, 'nuK': 1.5e15, 'nuH': 1.2e15}  # Example atom properties
grid = {'N_X': 100, 'N_Y': 100, 'N_Z': 100}  # Example grid dimensions
v_emit = 1.0e4  # Example velocity dispersion

photon = gen_photon_Gaussian(photon, v_emit, atom, grid)
print(photon)


/tmp/ipykernel_5835/70172251.py:38: RuntimeWarning: Mean of empty slice.
  y_new=em_new[y].mean()
/home/jin/.local/lib/python3.10/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'x': np.float64(0.40050298014320823), 'y': np.float64(0.5154869190777461), 'z': np.float64(-1.1915217759412957), 'ix': 51, 'iy': 51, 'iz': 51, 'x_s': np.float64(0.40050298014320823), 'y_s': np.float64(0.5154869190777461), 'z_s': np.float64(-1.1915217759412957), 'nu': 1500064418280585.0, 'line': 1}
